In [1]:
import pandas as pd
from datascience import *



In [ ]:
# 1. Load the data using the pyxlsb engine
# Note: Ensure you have pyxlsb installed (pip install pyxlsb)
df = pd.read_excel('BR Baseball Data 2021-2025.xlsb', engine='pyxlsb')

# 2. Filter for KC Royals players
# We use .copy() to ensure we have a clean, independent dataset
kc_royals_df = df[df['Team'] == 'KCR'].copy()

# 3. Convert to a datascience Table for better visual formatting
kc_royals_table = Table.from_df(kc_royals_df)

# 4. Display the table
kc_royals_count = len(kc_royals_table.rows)
print(f"Number of KC Royals players: {kc_royals_count}")
kc_royals_table.sort('Age', descending=True).show()

In [6]:
kc_royals_table.to_csv('kc_royals.csv')